<a href="https://colab.research.google.com/github/davideingersoll/Sample_API_Call_Sports_Model/blob/main/Playoff_Z_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Playoff Winner Sig Diffs</h1>
<p>This project will:
<ul>
  <li>Download all european player data from 2 tournaments</li>
  <li>Separate data into a corpus of winning team stats and a corpus of losing team stats</li>
  <li>Separate out the target team from both dataframes</li>
  <li>Compare target wins against other teams' wins</li>
  <li>Compare target losses against other teams' losses</li>
  <li>Separate out only stats that were significantly different from other teams during both wins and losses</li>
</ul>
</p>

In [1]:
import pandas as pd
import requests
from google.colab import userdata
from scipy import stats
import numpy as np
import time
from google.colab import files

#get your ballchasing token from the 'upload' tab on ballchasing, copy the key
#click the key on the left to open google secrets
#save your token as BALLCHASING_TOKEN
Ballchasing_Token = userdata.get("BALLCHASING_TOKEN")
head = {'Authorization': Ballchasing_Token,
        'Content-Type': 'application/json'}
groups_url = "https://ballchasing.com/api/groups?group="
replays_url = 'https://ballchasing.com/api/replays?group='
replay_url = 'https://ballchasing.com/api/replays/'

df = 'Empty'

groups =[
    'regional-1-pydu2pstsu',
    'regional-2-ubdrbyma2g'
    ]

replay_groups = []
replay_ids = []

def get_replays(group):
  time.sleep(0.5)
  r = requests.get(groups_url+group, headers=head)
  try:
    r = r.json()
  except:
    print(r)
  for sub_group in r['list']:
    if sub_group['direct_replays'] > 0:
      replay_groups.append(sub_group['id'])
    else:
      get_replays(sub_group['id'])



for group in groups:
  get_replays(group)

for r_group in replay_groups:
  time.sleep(0.5)
  r = requests.get(replays_url+r_group, headers=head).json()
  for replay in r['list']:
    replay_ids.append(replay['id'])

columns = ['Player', 'Team', 'ID', 'shots', 'shots_against', 'goals', 'goals_against', 'saves', 'assists', 'score', 'mvp', 'shooting_percentage', 'bpm', 'bcpm', 'avg_amount', 'amount_collected', 'amount_stolen', 'amount_collected_big', 'amount_stolen_big', 'amount_collected_small', 'amount_stolen_small', 'count_collected_big', 'count_stolen_big', 'count_collected_small', 'count_stolen_small', 'amount_overfill', 'amount_overfill_stolen', 'amount_used_while_supersonic', 'time_zero_boost', 'percent_zero_boost', 'time_full_boost', 'percent_full_boost', 'time_boost_0_25', 'time_boost_25_50', 'time_boost_50_75', 'time_boost_75_100', 'percent_boost_0_25', 'percent_boost_25_50', 'percent_boost_50_75', 'percent_boost_75_100', 'avg_speed', 'total_distance', 'time_supersonic_speed', 'time_boost_speed', 'time_slow_speed', 'time_ground', 'time_low_air', 'time_high_air', 'time_powerslide', 'count_powerslide', 'avg_powerslide_duration', 'avg_speed_percentage', 'percent_slow_speed', 'percent_boost_speed', 'percent_supersonic_speed', 'percent_ground', 'percent_low_air', 'percent_high_air', 'avg_distance_to_ball', 'avg_distance_to_ball_possession', 'avg_distance_to_ball_no_possession', 'avg_distance_to_mates', 'time_defensive_third', 'time_neutral_third', 'time_offensive_third', 'time_defensive_half', 'time_offensive_half', 'time_behind_ball', 'time_infront_ball', 'time_most_back', 'time_most_forward', 'goals_against_while_last_defender', 'time_closest_to_ball', 'time_farthest_from_ball', 'percent_defensive_third', 'percent_offensive_third', 'percent_neutral_third', 'percent_defensive_half', 'percent_offensive_half', 'percent_behind_ball', 'percent_infront_ball', 'percent_most_back', 'percent_most_forward', 'percent_closest_to_ball', 'percent_farthest_from_ball', 'inflicted', 'taken', 'win']
stats = []
for replay in replay_ids:
  time.sleep(0.5)
  r = requests.get(replay_url+replay, headers=head)
  print(r)
  r = r.json()

  for player in r['orange']['players']:
    temp = []
    temp.append(player['name'])
    try:
      temp.append(r['orange']['name'])
    except:
      temp.append('NA')
    temp.append(r['title']+'-'+r['groups'][0]['id'])
    for psc in player['stats']:
      for ps in list(player['stats'][psc].keys()):
        temp.append(player['stats'][psc][ps])
    orange_score = 0
    blue_score = 0
    for p in r['orange']['players']:
      orange_score = orange_score + p['stats']['core']['goals']
    for p in r['blue']['players']:
      blue_score = blue_score + p['stats']['core']['goals']
    if orange_score > blue_score:
      temp.append(1)
    else:
      temp.append(0)
    if len(temp) == 87:
      temp.insert(71, 0)
    stats.append(temp)
  for player in r['blue']['players']:
    temp = []
    temp.append(player['name'])
    try:
      temp.append(r['blue']['name'])
    except:
      temp.append("NA")
    temp.append(r['title']+'-'+r['groups'][0]['id'])
    for psc in player['stats']:
      for ps in list(player['stats'][psc].keys()):
        temp.append(player['stats'][psc][ps])
    orange_score = 0
    blue_score = 0
    for p in r['orange']['players']:
      orange_score = orange_score + p['stats']['core']['goals']
    for p in r['blue']['players']:
      blue_score = blue_score + p['stats']['core']['goals']
    if blue_score > orange_score:
      temp.append(1)
    else:
      temp.append(0)
    if len(temp) == 87:
      temp.insert(71, 0)
    stats.append(temp)

df = pd.DataFrame(stats, columns=columns)
df

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

,Player,Team,ID,shots,shots_against,goals,goals_against,saves,assists,score,...,percent_offensive_half,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball,inflicted,taken,win
0,ExoTiiK,TEAM VITALITY,EU P-I KC vs VIT G4 2025-01-12.21.21-monkey-bz...,1,9,0,4,1,0,225,...,38.690510,65.900680,34.099316,35.489273,35.671894,38.563385,33.145640,0,0,0
1,zen,TEAM VITALITY,EU P-I KC vs VIT G4 2025-01-12.21.21-monkey-bz...,2,9,0,4,1,0,213,...,38.741150,60.671154,39.328846,33.450010,39.902607,31.897734,33.754375,5,1,0
2,M0nkey M00n,TEAM VITALITY,EU P-I KC vs VIT G4 2025-01-12.21.21-monkey-bz...,0,9,0,4,1,0,142,...,35.128304,69.183740,30.816261,31.867296,23.588495,28.610563,33.936996,1,1,0
3,dralii,KARMINE CORP,EU P-I KC vs VIT G4 2025-01-12.21.21-monkey-bz...,3,3,2,0,2,2,512,...,40.838990,75.876150,24.123850,27.149597,38.380764,34.058743,32.262974,0,1,1
4,Atow.,KARMINE CORP,EU P-I KC vs VIT G4 2025-01-12.21.21-monkey-bz...,4,3,1,0,0,2,408,...,37.578240,75.542114,24.457893,30.132400,39.567800,39.446056,36.067570,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,Mittaen,STARTPOINT,2025-01-31.17.18 yujin Private Win-red-xvuwyoyf3m,5,5,1,0,0,0,298,...,38.928062,78.337320,21.662682,29.749138,27.915073,41.250893,24.340204,1,0,1
1724,acro.,STARTPOINT,2025-01-31.17.18 yujin Private Win-red-xvuwyoyf3m,2,5,0,0,1,0,255,...,34.677170,71.060745,28.939250,38.732940,37.085392,27.728558,44.825764,2,1,1
1725,Kash,REDEMPTION,2025-01-31.17.18 yujin Private Win-red-xvuwyoyf3m,2,11,0,2,2,0,339,...,37.535940,62.563934,37.436070,33.945724,37.862537,31.645372,34.847214,0,0,0
1726,Toxiic,REDEMPTION,2025-01-31.17.18 yujin Private Win-red-xvuwyoyf3m,2,11,0,2,2,0,248,...,41.404526,65.954130,34.045860,37.178650,28.225931,30.308681,35.282413,2,3,0


<h2>Win Comparison</h2>
<p>Z tests KC wins against all other winners and only keeps stats that are significantly different</p>

In [2]:
from scipy import stats
import numpy as np

#team to be targetted for comparison
target_name = "KARMINE CORP"


target = df[df['Team']==target_name]
target = target.drop(['Player','Team','ID','mvp'], axis=1)
target_wins = target[target['win']==1]
target_losses = target[target['win']==0]
population = df[df['Team']!=target_name]
population = population.drop(['Player','Team','ID','mvp'], axis=1)
population_wins = population[population['win']==1]
population_losses = population[population['win']==0]

target_wins, target_losses = target_wins.drop("win", axis=1), target_losses.drop('win', axis=1)
population_wins, population_losses = population_wins.drop('win', axis=1), population_losses.drop('win', axis=1)

win_compare = []
for item in target_wins.columns.values:
  z = (target_wins[item].mean() - population_wins[item].mean()) / (population_wins[item].std()/target_wins[item].count()**0.5)
  p = 2 * stats.norm.cdf(-np.absolute(z))
  win_compare.append([item, z, p, p < 0.05, target_wins[item].mean(), population_wins[item].mean()])
win_compare = pd.DataFrame(win_compare, columns=['stat_name','z_stat_W','p_value','sig_dif','target_mean_W','pop_mean_W'])
win_compare = win_compare[win_compare['sig_dif']].reset_index(drop=True)

win_compare

,stat_name,z_stat_W,p_value,sig_dif,target_mean_W,pop_mean_W
0,shots_against,-3.971776,7.133889e-05,True,6.285714,7.351020
1,goals_against,-2.038862,4.146381e-02,True,0.976190,1.142857
2,bpm,7.551218,4.312057e-14,True,441.023810,413.896599
3,bcpm,6.140146,8.244551e-10,True,450.578890,426.659486
4,avg_amount,3.886385,1.017480e-04,True,50.561905,48.961850
5,amount_collected_small,4.349344,1.365452e-05,True,876.380952,801.668027
6,amount_stolen_small,3.666738,2.456646e-04,True,288.500000,257.469388
7,count_collected_small,5.453469,4.939654e-08,True,77.174603,69.311565
8,count_stolen_small,4.644250,3.413148e-06,True,25.373016,22.190476
9,time_zero_boost,-2.459630,1.390802e-02,True,38.589603,41.941918


<h2>Loss Comparison</h2>
<p>Z tests KC losses against all other losers and only keeps stats that are significantly different</p>

In [3]:
loss_compare = []
for item in target_losses.columns.values:
  z = (target_losses[item].mean() - population_losses[item].mean()) / (population_losses[item].std()/target_losses[item].count()**0.5)
  p = 2 * stats.norm.cdf(-np.absolute(z))
  loss_compare.append([item, z, p, p < 0.05, target_losses[item].mean(), population_losses[item].mean()])
loss_compare = pd.DataFrame(loss_compare, columns=['stat_name','z_stat_L','p_value','sig_dif','target_mean_L','pop_mean_L'])
loss_compare = loss_compare[loss_compare['sig_dif']].reset_index(drop=True)

loss_compare

,stat_name,z_stat_L,p_value,sig_dif,target_mean_L,pop_mean_L
0,shots_against,-4.180889,2.903721e-05,True,7.333333,10.178571
1,goals_against,-3.156400,1.597298e-03,True,2.222222,3.071429
2,bpm,3.541573,3.977493e-04,True,440.185185,412.463095
3,bcpm,3.512127,4.445351e-04,True,453.308585,422.953163
4,avg_amount,4.172446,3.013469e-05,True,51.685556,47.987202
5,amount_stolen,2.455143,1.408286e-02,True,683.222222,575.808333
6,amount_stolen_small,3.376563,7.339757e-04,True,306.629630,249.380952
7,count_stolen_small,3.285930,1.016461e-03,True,26.222222,21.617857
8,amount_overfill,2.065541,3.887189e-02,True,412.814815,348.352381
9,time_zero_boost,-3.001195,2.689219e-03,True,34.616667,43.477667


<h2>Final Comparison</h2>
<p>Below is the table that indicates the stat categories that KC performed significantly differently than the body of teams during both wins and losses. KC team score and population are displayed for all stats to use as a reference</p>

In [4]:
common_difs = [x for x in list(win_compare['stat_name']) if x in list(loss_compare['stat_name'])]


common_difs_table = win_compare[win_compare.stat_name.isin(common_difs)][['stat_name','z_stat_W','target_mean_W','pop_mean_W']].\
  merge(loss_compare[loss_compare.stat_name.isin(common_difs)][['stat_name','z_stat_L','target_mean_L','pop_mean_L']], on='stat_name', how='inner')
common_difs_table

,stat_name,z_stat_W,target_mean_W,pop_mean_W,z_stat_L,target_mean_L,pop_mean_L
0,shots_against,-3.971776,6.285714,7.351020,-4.180889,7.333333,10.178571
1,goals_against,-2.038862,0.976190,1.142857,-3.156400,2.222222,3.071429
2,bpm,7.551218,441.023810,413.896599,3.541573,440.185185,412.463095
3,bcpm,6.140146,450.578890,426.659486,3.512127,453.308585,422.953163
4,avg_amount,3.886385,50.561905,48.961850,4.172446,51.685556,47.987202
5,amount_stolen_small,3.666738,288.500000,257.469388,3.376563,306.629630,249.380952
6,count_stolen_small,4.644250,25.373016,22.190476,3.285930,26.222222,21.617857
7,time_zero_boost,-2.459630,38.589603,41.941918,-3.001195,34.616667,43.477667
8,time_full_boost,-6.389030,32.959127,42.061905,-2.154281,32.671111,38.768702
9,avg_speed,9.728942,1608.039683,1549.798639,4.706292,1624.851852,1557.452381


In [5]:
#downloads table. commented out to not run every time file is run
from google.colab import files

common_difs_table.to_excel('H2H_Z_Tests.xlsx', index=False)
#files.download('H2H_Z_Tests.xlsx')